In [1]:
import os
import sys
import time
import pandas as pd
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
import multiprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, Callback, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.utils import multi_gpu_model
from sklearn.metrics.ranking import roc_auc_score
from sklearn.model_selection import train_test_split
from common.utils import *

Using TensorFlow backend.


In [2]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.1.4
Numpy:  1.14.1
Tensorflow:  1.4.0
tensorflow
channels_first
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
# User-set
# Note if NUM_GPUS > 1 then MULTI_GPU = True and ALL GPUs will be used
# Set below to affect batch-size
# E.g. 1 GPU = 64, 2 GPUs = 64*2, 4 GPUs = 64*4
# Note that the effective learning-rate will be decreased this way
NUM_GPUS = 2 # Scaling factor for batch
NUM_CPUS = multiprocessing.cpu_count()
print(NUM_CPUS)

12


In [5]:
# Globals
CLASSES = 14
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001  # Effective learning-rate will decrease as BATCHSIZE rises
EPOCHS = 5
BATCHSIZE = 64*NUM_GPUS
#IMAGENET_RGB_MEAN = [0.485, 0.456, 0.406]
#IMAGENET_RGB_SD = [0.229, 0.224, 0.225]
TOT_PATIENT_NUMBER = 30805  # From data

In [6]:
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 625 ms, sys: 244 ms, total: 869 ms
Wall time: 869 ms


In [8]:
#####################################################################################################
## Data Loading

In [43]:
class XrayData():
    
    def __init__(self, img_dir, lbl_file, patient_ids, 
                 width=WIDTH, height=HEIGHT, batch_size=BATCHSIZE, num_classes=CLASSES,
                 shuffle=True, seed=None, augment=False):
        
        self.patient_ids = patient_ids
        self.lbl_file = lbl_file
        
        # Hack for flow_from_directory to work, give it path above
        self.child_path  = os.path.split(img_dir)[-1]
        self.parent_path =  img_dir.replace(self.child_path,'')
        
        # Create ImageDataGenerator with DenseNet pre-processing
        # imagenet_utils.preprocess_input(x, data_format, mode='torch')
        if augment:
            datagen = ImageDataGenerator(
                horizontal_flip=True,
                # Best match to?
                # transforms.RandomResizedCrop(size=WIDTH),
                zoom_range=0.2,  
                rotation_range=10,
                preprocessing_function=preprocess_input)
        else:
             datagen = ImageDataGenerator(preprocessing_function=preprocess_input)    

        # Create flow-from-directory
        flowgen = datagen.flow_from_directory(
            directory=self.parent_path,  # hack: this is one directory up
            target_size=(width, height),
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            class_mode='binary')    
        
        # Override previously created classes variables
        # filenames, classes
        flowgen.filenames, flowgen.classes = self._override_classes()
        # number of files
        flowgen.n = len(flowgen.filenames)
        # number of classes (not sure if this last one needed)
        flowgen.num_classes = num_classes
        
        self.generator = flowgen
        print("Loaded {} labels and {} images".format(len(self.generator.classes), 
                                                      len(self.generator.filenames)))

    
    def _override_classes(self):
        # Read labels-csv
        df = pd.read_csv(self.lbl_file)

        # Split labels on unfiltered data
        df_label = df['Finding Labels'].str.split(
            '|', expand=False).str.join(sep='*').str.get_dummies(sep='*')
        
        # Filter by patient-ids (both)
        df_label['Patient ID'] = df['Patient ID']
        
        df_label = df_label[df_label['Patient ID'].isin(self.patient_ids)]
        df = df[df['Patient ID'].isin(self.patient_ids)]

        # Remove unncessary columns
        df_label.drop(['Patient ID','No Finding'], axis=1, inplace=True)
        
        # List of images       
        img_locs =  df['Image Index'].map(lambda im: os.path.join(self.child_path, im)).values
        labels = df_label.values
        # Return new file-names and labels
        return img_locs, labels

In [44]:
# Training / Valid / Test split (70% / 10% / 20%)
train_set, other_set = train_test_split(
    range(1,TOT_PATIENT_NUMBER+1), train_size=0.7, test_size=0.3, shuffle=False)
valid_set, test_set = train_test_split(other_set, train_size=1/3, test_size=2/3, shuffle=False)
print("train:{} valid:{} test:{}".format(
    len(train_set), len(valid_set), len(test_set)))

train:21563 valid:3080 test:6162


In [45]:
train_dataset = XrayData(IMAGE_FOLDER, LABEL_FILE, train_set, augment=True).generator

Found 112120 images belonging to 1 classes.
Loaded 87306 labels and 87306 images


In [46]:
valid_dataset = XrayData(IMAGE_FOLDER, LABEL_FILE, valid_set, shuffle=False).generator
test_dataset = XrayData(IMAGE_FOLDER, LABEL_FILE, test_set, shuffle=False).generator

Found 112120 images belonging to 1 classes.
Loaded 7616 labels and 7616 images
Found 112120 images belonging to 1 classes.
Loaded 17198 labels and 17198 images


In [13]:
#####################################################################################################
## Helper Functions

In [14]:
def get_symbol(model_name='densenet121', out_features=CLASSES):
    # Recommended to instantiate base model on CPU
    # https://keras.io/utils/#multi_gpu_model
    # Yet another Keras hack ...
    with tensorflow.device('/cpu:0'):
        if model_name == 'densenet121':
            model = DenseNet121(input_shape=(3, 224, 224), weights='imagenet', include_top=False, pooling='avg')
        else:
            raise ValueError("Unknown model-name")
        # Add classifier to model FC-14
        classifier = Dense(out_features, activation='sigmoid')(model.output)
        model = Model(inputs=model.input, outputs=classifier)
    return model

In [15]:
def init_symbol(sym, lr=LR):
    # BCE Loss since classes not mutually exclusive + Sigmoid FC-layer
    sym.compile(
        loss = "binary_crossentropy",
        optimizer = Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=None))
    # Callbacks
    sch = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)
    #This doesnt work with Keras multi-gpu
    #Don't want to add another hack to get it fixed
    #chp = ModelCheckpoint('best_chexnet.pth.hdf5', monitor='val_loss', save_weights_only=False)
    callbacks = [sch]
    return sym, callbacks

In [16]:
def compute_roc_auc(data_gt, data_pd, full=True, classes=CLASSES):
    roc_auc = []
    for i in range(classes):
        roc_auc.append(roc_auc_score(data_gt[:, i], data_pd[:, i]))
    print("Full AUC", roc_auc)
    roc_auc = np.mean(roc_auc)
    return roc_auc

In [17]:
#####################################################################################################
## Train CheXNet

In [18]:
%%time
# Load symbol
chexnet_sym = get_symbol()

CPU times: user 18.8 s, sys: 1.37 s, total: 20.2 s
Wall time: 19.8 s


In [19]:
%%time
# Load optimiser, loss
multi_gpu_sym = multi_gpu_model(chexnet_sym, gpus=NUM_GPUS)
model, callbacks = init_symbol(multi_gpu_sym)

CPU times: user 23.9 s, sys: 419 ms, total: 24.3 s
Wall time: 24 s


In [21]:
%%time
# Training loop: 31m38s
model.fit_generator(train_dataset,
                    epochs=EPOCHS,
                    verbose=1,
                    callbacks=callbacks,
                    workers=NUM_CPUS,  # Num of CPUs since multiprocessing
                    use_multiprocessing=True,  # Faster than with threading
                    validation_data=valid_dataset,
                    max_queue_size=20)  # Default is 10 (most prob no difference)

Epoch 1/5
683/683 [==============================] - 405s 593ms/step - loss: 0.2037 - val_loss: 0.1523
Epoch 2/5
683/683 [==============================] - 366s 536ms/step - loss: 0.1517 - val_loss: 0.1454
Epoch 3/5
683/683 [==============================] - 368s 539ms/step - loss: 0.1473 - val_loss: 0.1483
Epoch 4/5
683/683 [==============================] - 368s 538ms/step - loss: 0.1443 - val_loss: 0.1409
Epoch 5/5
683/683 [==============================] - 368s 540ms/step - loss: 0.1417 - val_loss: 0.1397
CPU times: user 59min 36s, sys: 17min 6s, total: 1h 16min 43s
Wall time: 31min 38s


In [49]:
#####################################################################################################
## Test CheXNet

In [50]:
# Load model for testing
# Currently multi-GPU checkpointing is broken on Keras
# For now use in-RAM model

In [51]:
%%time
## Evaluate
# AUC: 0.8174
y_guess = model.predict_generator(test_dataset, workers=NUM_CPUS)

CPU times: user 6min 7s, sys: 1min 23s, total: 7min 30s
Wall time: 1min 46s


In [52]:
print("Validation AUC: {0:.4f}".format(compute_roc_auc(test_dataset.classes, y_guess)))

Full AUC [0.8111778411803284, 0.8640277398006696, 0.8005941321547805, 0.8917648363461342, 0.8820978933695289, 0.9348748895066568, 0.7213946018481607, 0.8607471465175869, 0.6317882959983487, 0.85477878143565, 0.7437874984083546, 0.8049289766045833, 0.7531004915756688, 0.8884164796734438]
Validation AUC: 0.8174
